<a href="https://colab.research.google.com/github/jx-dohwan/Aiffel_EGLMS_Project/blob/main/%5BExp_13%5DMovie_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

## 1. Import 및 라이브러리 다운로드

In [2]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 399 kB/s 


In [4]:
import numpy as np
import scipy
import implicit
import pandas as pd


## 2. 데이터 준비와 전처리

In [5]:
rating_file_path = '/content/drive/MyDrive/인공지능/아이펠/ES/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [7]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [8]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [9]:
movie_file_path= '/content/drive/MyDrive/인공지능/아이펠/ES/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 3. 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
movies['title'].nunique()

3883

In [11]:
movies['genre'].nunique()

301

In [12]:
movies['movie_id'].nunique()

3883

In [13]:
genre_count = movies.groupby('genre')['movie_id'].count()
genre_count.sort_values(ascending=False).head(30)

genre
Drama                      843
Comedy                     521
Horror                     178
Comedy|Drama               162
Comedy|Romance             142
Drama|Romance              134
Documentary                116
Thriller                   101
Action                      65
Drama|Thriller              63
Action|Thriller             48
Children's|Comedy           47
Crime|Drama                 44
Drama|War                   43
Romance                     40
Action|Drama                39
Animation|Children's        35
Comedy|Drama|Romance        34
Horror|Sci-Fi               33
Western                     33
Horror|Thriller             32
Comedy|Horror               31
Adventure|Children's        30
Action|Sci-Fi               28
Sci-Fi                      27
Crime                       26
Action|Adventure            25
Action|Comedy               22
Action|Adventure|Sci-Fi     21
Crime|Thriller              21
Name: movie_id, dtype: int64

In [14]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = movies.groupby('movie_id')['genre'].count()
user_count.describe()

count    3883.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: genre, dtype: float64

### 4. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [18]:
movies['title'].head(30)

0                                      Toy Story (1995)
1                                        Jumanji (1995)
2                               Grumpier Old Men (1995)
3                              Waiting to Exhale (1995)
4                    Father of the Bride Part II (1995)
5                                           Heat (1995)
6                                        Sabrina (1995)
7                                   Tom and Huck (1995)
8                                   Sudden Death (1995)
9                                      GoldenEye (1995)
10                       American President, The (1995)
11                   Dracula: Dead and Loving It (1995)
12                                         Balto (1995)
13                                         Nixon (1995)
14                              Cutthroat Island (1995)
15                                        Casino (1995)
16                         Sense and Sensibility (1995)
17                                    Four Rooms

In [19]:
my_favorite = ['Nixon' , 'Heat' ,'Toy Story' ,'City of Lost Children, The' ,'Sudden Death']

my_playlist = pd.DataFrame({'movie_id' : ['dohwan'] *5, 'genre': my_favorite,'ratings':[30]*5})

if not data.isin({''})

### 5. CSR matrix를 직접 만들어 봅시다.

## 6. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

## 7. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

## 8. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

## 9. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

## 10. 회고